<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Изучение-файла" data-toc-modified-id="Изучение-файла-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Изучение файла</a></span><ul class="toc-item"><li><span><a href="#Загрузка-необходимых-для-решения-библиотек:" data-toc-modified-id="Загрузка-необходимых-для-решения-библиотек:-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Загрузка необходимых для решения библиотек:</a></span></li><li><span><a href="#Ознакомление-с-данными" data-toc-modified-id="Ознакомление-с-данными-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Ознакомление с данными</a></span></li></ul></li><li><span><a href="#Разделение-данных-на-выборки" data-toc-modified-id="Разделение-данных-на-выборки-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Разделение данных на выборки</a></span></li><li><span><a href="#Исследуйте-модели" data-toc-modified-id="Исследуйте-модели-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Исследуйте модели</a></span><ul class="toc-item"><li><span><a href="#Модель-Random-Forest" data-toc-modified-id="Модель-Random-Forest-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Модель Random Forest</a></span></li><li><span><a href="#Модель-Decision-Tree" data-toc-modified-id="Модель-Decision-Tree-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Модель Decision Tree</a></span></li><li><span><a href="#Модель-Logistic-Regression" data-toc-modified-id="Модель-Logistic-Regression-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Модель Logistic Regression</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Проверка-моделей-на-тестовой-выборке" data-toc-modified-id="Проверка-моделей-на-тестовой-выборке-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка моделей на тестовой выборке</a></span></li></ul></div>

# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Изучение файла

### Загрузка необходимых для решения библиотек:

In [1]:
#для работы с таблицами
import pandas as pd

#для разделения таблицы на выборки
from sklearn.model_selection import train_test_split

#три различные модели решения
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

#для вычисления точности предсказания модели
from sklearn.metrics import accuracy_score

#модуль, который выводит на консоль динамически обновляемый индикатор выполнения
from tqdm import tqdm

### Ознакомление с данными

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:  
* сalls — количество звонков,  
* minutes — суммарная длительность звонков в минутах,  
* messages — количество sms-сообщений,  
* mb_used — израсходованный интернет-трафик в Мб,  
* is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

Откроем и изучим файл с данными:

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Оценим информацию о столбцах таблицы:

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Как видно из информации выше, в таблице нет пропусков. Однако, столбец massages принадлежит к типу данных "число с плавающей точкой", что является некорректным: количество sms-сообщений не может быть дробным.  

Исправим тип данных в этом столбце на целочисленный:

In [4]:
df['messages'] = df['messages'].astype('int')

Оценим количество полных дубликатов в таблице:

In [5]:
print('Число полных дубликатов в таблице равно:', df.duplicated().sum())

Число полных дубликатов в таблице равно: 0


Оценим разброс значений в столбцах таблицы:

In [6]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


Из информации выше видно, что ряд пользователей не используют или функционал звонков, или sms-сообщений, или интернет в рамках тарифа оператора связи.

Оценим корреляцию данных в столбцах таблицы:

In [7]:
df.corr()

,calls,minutes,messages,mb_used,is_ultra
calls,1.000000,0.982083,0.177385,0.286442,0.207122
minutes,0.982083,1.000000,0.173110,0.280967,0.206955
messages,0.177385,0.173110,1.000000,0.195721,0.203830
mb_used,0.286442,0.280967,0.195721,1.000000,0.198568
is_ultra,0.207122,0.206955,0.203830,0.198568,1.000000


Из данных выше видно, что взаимное влияние количества совершенных звонков и количества потраченных минут является крайне сильным, то есть обнаружено 2 параметра с высокой корреляцией.  
Для возможного улучшения работы моделей предлагается рассмотреть следующий подход- проводить эксперименты по удалению этих признаков: сначала обучить модель на всех признаках и измерить качество, далее по очереди удалять по одному из этих признаков и снова измерять качество.

## Разделение данных на выборки

Разделим исходные данные на обучающую, валидационную и тестовую выборки в соотношении 3:1:1.

Перед этим выделим:  
* _целевой признак_ - (тип тарифа) столбец is_ultra;
* _матрицу признаков_ - столбцы calls, minutes, messages, mb_used.

In [8]:
#выделяем целевой признак, оставляя в нем только столбец с категориальным признаком
target = df['is_ultra']
#выделяем матрицу признаков, вычитая из таблицы целевой признак (удаляя столбец)
features = df.drop(['is_ultra'], axis=1)

Выделим обучающую выборку, она составит 60% от общего числа данных:

In [9]:
(features_train, 
 features_part_2, 
 target_train, 
 target_part_2) = train_test_split(features, 
                                  target, 
                                  test_size=0.4, 
                                  random_state=12345) 
# отделим 40% данных для дальнейшего разбиения на валидационную и тестовую выборки
# при этом 60% данных останется для обучения модели в переменных features_train и target_train
# features_train - тренировочная матрица признаков
# target_train - тренировочный целевой признак

# для разделения данных используется функция train_test_split библиотеки sklearn.model_selection
# random_state=12345 - произвольное фиксированное состояние модели

Выделим валидационные и тестовые выборки:

In [10]:
(features_valid, 
 features_test, 
 target_valid, 
 target_test) = train_test_split(features_part_2, 
                                  target_part_2, 
                                  test_size=0.5, 
                                  random_state=12345)
# разделим оставшиеся данные пополам: на валидационную и тестовую выборки

Таким образом мы получили:  
1) для непосредственного обучения модели - обучающую  выборку
* features_train - тренировочную матрицу признаков
* target_train - тренировочный целевой признак 

2) для выбора наилучшей модели машинного обучения - проверочную (валидационную) выборку  
* features_valid - валидационную матрицу признаков
* target_valid - валидационый целевой признак  

3) для оценки качества модели - тестовую выборку  
* features_test - тестовая матрица признаков  
* target_test - тестовый целевой признак

## Исследуйте модели

### Модель Random Forest

По сути, Random Forest является композицией (ансамблем) множества решающих деревьев, что позволяет снизить проблему переобучения и повысить точность в сравнении с одним деревом. Прогноз получается в результате агрегирования ответов множества деревьев. Тренировка деревьев происходит независимо друг от друга (на разных подмножествах), что решает проблему построения одинаковых деревьев на одном и том же наборе данных.  
[Источник](https://habr.com/ru/post/320726/)

In [11]:
# лучшая модель "случайного леса"
best_RF = None
# точность лучшей модели
best_accuracy_RF = 0
# количество деревьев лучшей модели
best_est_RF = 0
# глубина лучшей модели
best_depth_RF = 0

# цикл от 1 до 50 с шагом в 10 для количества деревьев
for est in tqdm(range(1,51, 10)):
    #глубина каждого дерева от 2 до 10 с шагом 1
    for depth in range(2, 10):
        # объявляем модель Random Forest с заданным количеством деревьев, заданной глубиной
        model_RF = RandomForestClassifier(random_state = 12345, 
                                          n_estimators = est,
                                          max_depth = depth)
        # обучаем модель на тренировочной выборке
        model_RF.fit(features_train,target_train)
        # получаем предсказания модели на валидационной выборке
        prediction_valid_RF = model_RF.predict(features_valid)
        #считаем точность предсказания модели
        accuracy_RF = accuracy_score(prediction_valid_RF, target_valid)
        #обновляем данные о лучшей модели
        if best_accuracy_RF < accuracy_RF:
            best_RF = model_RF
            best_depth_RF = depth
            best_est_RF = est
            best_accuracy_RF = accuracy_RF

# выводим точность лучшей модели
print(f"Точность лучшей модели Random Forest: {best_accuracy_RF}")
print(f"Глубина лучшей модели Random Forest: {best_depth_RF}")
print(f"Количество деревьев лучшей модели Random Forest: {best_est_RF}")

100%|██████████| 5/5 [00:02<00:00,  1.73it/s]

Точность лучшей модели Random Forest: 0.807153965785381
Глубина лучшей модели Random Forest: 8
Количество деревьев лучшей модели Random Forest: 41


### Модель Decision Tree

В основе популярных алгоритмов построения дерева решений лежит принцип жадной максимизации прироста информации – на каждом шаге выбирается тот признак, при разделении по которому прирост информации оказывается наибольшим. Дальше процедура повторяется рекурсивно, пока энтропия не окажется равной нулю или какой-то малой величине.  
[Источник](https://habr.com/ru/company/ods/blog/322534/)

In [12]:
# лучшая модель "дерева решений"
best_DT = None
# точность лучшей модели
best_accuracy_DT = 0
# глубина дерева для лучшей модели
best_depth_DT = 0
# цикл от 2 до 50 с шагом в 10 для глубины дерева
for depth in tqdm(range(2,52,10)):
    # объявляем модель Decision Tree с заданной глубиной
    model_DT = DecisionTreeClassifier(random_state = 12345, max_depth = depth)
    # обучаем модель на тренировочной выборке
    model_DT.fit(features_train,target_train)
    # получаем предсказания модели на валидационной выборке
    prediction_valid_DT = model_DT.predict(features_valid)
    #считаем точность предсказания модели
    accuracy_DT = accuracy_score(prediction_valid_DT, target_valid)
    #обновляем данные о лучшей модели
    if best_accuracy_DT < accuracy_DT:
        best_DT = model_DT
        best_accuracy_DT = accuracy_DT
        best_depth_DT = depth

# выводим точность лучшей модели
print(f"Точность лучшей модели Decision Tree: {best_accuracy_DT}")
print(f"Глубина лучшей модели Decision Tree: {best_depth_DT}")

100%|██████████| 5/5 [00:00<00:00, 68.67it/s]

Точность лучшей модели Decision Tree: 0.7822706065318819
Глубина лучшей модели Decision Tree: 2


### Модель Logistic Regression

Логистическая регрессия – это алгоритм классификации, используемый для отнесения наблюдений к дискретному набору классов.  
[Источник](https://www.helenkapatsa.ru/loghistichieskaia-rieghriessiia/)

In [13]:
model_LR = LogisticRegression(random_state = 12345)
model_LR.fit(features_train,target_train)
prediction_valid_LR = model_LR.predict(features_valid)
accuracy_LR = accuracy_score(prediction_valid_LR, target_valid)

print(f"Точность лучшей модели Logistic Regression: {accuracy_LR}")

Точность лучшей модели Logistic Regression: 0.7107309486780715


### Вывод

Основываясь на показателе точности (accuracy), делаем вывод, что наиболее качественные результаты прогнозирует моедель Random Forest.  
Точность лучшей модели Random Forest: 0.807153965785381

## Проверка моделей на тестовой выборке

In [14]:
# получаем предсказания моделей на тестовой выборке
prediction_test_RF = best_RF.predict(features_test)
prediction_test_DT = best_DT.predict(features_test)
prediction_test_LR = model_LR.predict(features_test)

In [15]:
#считаем точность предсказания моделей
accuracy_RF_test = accuracy_score(prediction_test_RF, target_test)
accuracy_DT_test = accuracy_score(prediction_test_DT, target_test)
accuracy_LR_test = accuracy_score(prediction_test_LR, target_test)
# выводим точность моделей для тестовой выборки
print(f"Точность модели Random Forest на тестовой выборке: {accuracy_RF_test}")
print(f"Точность модели Decision Tree на тестовой выборке: {accuracy_DT_test}")
print(f"Точность модели Logistic Regression на тестовой выборке: {accuracy_LR_test}")

Точность модели Random Forest на тестовой выборке: 0.7978227060653188
Точность модели Decision Tree на тестовой выборке: 0.7744945567651633
Точность модели Logistic Regression на тестовой выборке: 0.6842923794712286
